<a href="https://colab.research.google.com/github/2yeonsong/MachineLearning/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 검증 세트

In [3]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
# 결정 트리 이용
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [10]:
# 훈련 세트와 검증 세트의 평가 점수를 이용하여 최적화
# 최적화 완료 후 테스트 세트로 모델을 최종 평가
print(dt.score(test_input, test_target))

0.8569230769230769


In [15]:
# 최적화 하기(이게 맞나?)
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.7804502597652492
0.7776923076923077


In [16]:
# 교차 검증

In [17]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02476501, 0.01957679, 0.02516723, 0.03118396, 0.01530242]), 'score_time': array([0.00281358, 0.00794888, 0.00203943, 0.00201154, 0.00226736]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [19]:
# test_score의 평균값(검증값의 평균?)
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [23]:
# 주의(책 247쪽)
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(StratifiedKFold())
print(scores)

import numpy as np

print(np.mean(scores['test_score']))

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
{'fit_time': array([0.01376128, 0.00824571, 0.00959992, 0.00844383, 0.00785804]), 'score_time': array([0.00276804, 0.00152922, 0.0014534 , 0.00140142, 0.00137639]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [24]:
# 그리드 서치(그냥 이런게 있다~)
# 하이퍼파라미터 탐색과 교차 검정을 한 번에 수행
# cross_vaildate()함수를 호출할 필요가 없음

In [25]:
# 예제 : 결정 트리 모델에서 min_impurity_decrease 최적값 찾기
# GridSearxhVC 클래스를 import하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
print(gs)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})


In [27]:
# 중요
# cv의 기본값은 5이므로 params에서 설정한 5개의 min_impurity_decrease 별로 결정 트리를 5번 실행
# 교차 검증 어쩌구가 각각 5번이고 [0.0001, 0.0002, 0.0003, 0.0004, 0.0005] 이게 다섯개라서 25번?!
# 즉, 25번의 결정 트리 모델을 훈련
# n_jobs : 병렬 실행에서 사용할 CPU 코어의 수이며, -1이면 모든 코어를 사용함
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [28]:
print(gs.cv_results_)

{'mean_fit_time': array([0.01050005, 0.00824618, 0.00813046, 0.00748329, 0.01040072]), 'std_fit_time': array([0.0010171 , 0.00011732, 0.00041046, 0.00036287, 0.0050988 ]), 'mean_score_time': array([0.00213742, 0.00165844, 0.00216293, 0.00169916, 0.00166535]), 'std_score_time': array([2.95066895e-04, 5.43736556e-05, 8.76868049e-04, 1.11328804e-04,
       3.35418354e-04]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216,

In [29]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8681929740134745


In [ ]:
# 중요(책 250쪽)
# 그리드 서치를 이용한 최적화 과정 정리
# 1. 탐색할 매개변수 지정
# 2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾음
# 3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델 훈련

In [31]:
# 예제 : 결정 트리 모델에서 min_impurity_decrease, max_depth, main_sample_slpit 최적값 찾기
# (사진 있음)

In [32]:
# 랜덤 서치
# 싸이파이(scipy)

In [37]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([3, 2, 0, 9, 2, 8, 2, 7, 4, 2])

In [38]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98,  96,  90, 101,  94, 106, 102, 101, 105, 107]))

In [39]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.24206453, 0.51371088, 0.34161502, 0.61695095, 0.31047216,
       0.56542781, 0.19862306, 0.73424997, 0.1166199 , 0.20335307])

In [40]:
# 예제 : 결정 트리 모델에서 min_impurity_decrease, max_depth, main_sample_slpit 최적값 찾기
# min_sample_leaf : 리프 노드가 되기 위한 최소 샘플의 수
# 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작으면 분할 안됨
# 샘플링 횟수는 n_iter 매개변수에 지정
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [41]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [42]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


In [ ]:
# 주의
# 교차검증, 그리드 서치, 랜덤 서치는 모델을 훈련하는 것이 아니라 검증하는 것임
# 즉, 적절한 하이퍼파라미터를 찾는데 사용해야 함
# 찾은 하이퍼파라미터를 훈련(fit())을 하여 모델을 만들어야 함